In [19]:
import numpy as py
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [20]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [21]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)


In [22]:
image, label = train_data[0]

In [23]:
image.size()

torch.Size([3, 32, 32])

In [24]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [25]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5) # 32 - 5/1 = 27 + 1 = 28 -> (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2) # (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) # 14 - 5 / 1 + 1 = 10 -> (24, 10, 10) -> (24, 5, 5) -> Flatten (24 * 5 *5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # output must be 10

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [26]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [27]:
for epoch in range(50):
    print(f'training epoch {epoch}...')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'loss: {running_loss / len(train_loader):.4f}')

training epoch 0...
loss: 2.2106
training epoch 1...
loss: 1.7635
training epoch 2...
loss: 1.5178
training epoch 3...
loss: 1.3984
training epoch 4...
loss: 1.2959
training epoch 5...
loss: 1.1967
training epoch 6...
loss: 1.1193
training epoch 7...
loss: 1.0606
training epoch 8...
loss: 1.0086
training epoch 9...
loss: 0.9640
training epoch 10...
loss: 0.9220
training epoch 11...
loss: 0.8858
training epoch 12...
loss: 0.8554
training epoch 13...
loss: 0.8224
training epoch 14...
loss: 0.7943
training epoch 15...
loss: 0.7673
training epoch 16...
loss: 0.7406
training epoch 17...
loss: 0.7185
training epoch 18...
loss: 0.6888
training epoch 19...
loss: 0.6677
training epoch 20...
loss: 0.6447
training epoch 21...
loss: 0.6237
training epoch 22...
loss: 0.6025
training epoch 23...
loss: 0.5830
training epoch 24...
loss: 0.5598
training epoch 25...
loss: 0.5372
training epoch 26...
loss: 0.5193
training epoch 27...
loss: 0.4979
training epoch 28...
loss: 0.4836
training epoch 29...
los

In [34]:
torch.save(net.state_dict(), 'trained_net.pth')

In [35]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [36]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, pred = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

Accuracy: 66.02%


In [37]:
base_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(img_path):
    img = Image.open(img_path)
    img = base_transform(img)
    img = img.unsqueeze(0)
    return img

img_path = ['Dog_Breeds.jpg', 'images.jpg', 'Cat03.jpg']
images = [load_image(image) for image in img_path]

net.eval()
with torch.no_grad():
    for img in images:
        output = net(img)
        _, pred = torch.max(output, 1)
        print(f'Prediction: {class_names[pred.item()]}')

Prediction: dog
Prediction: car
Prediction: cat
